## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [19]:
from datasets import load_dataset, get_dataset_split_names
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")
print(dataset)

train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

print(train_dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})


#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [24]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert/distilbert-base-uncased")

sample_texts = [train_dataset[i]['text'] for i in range(3)]
print("\nSample Texts:")
for text in sample_texts:
    print("-", text)

inputs = tokenizer(sample_texts, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

print("\nOutput Keys:", outputs.keys())
print("Last Hidden State shape:", outputs.last_hidden_state.shape)

cls_embeddings = outputs.last_hidden_state[:, 0, :]
print("\n[CLS] Token Embeddings (shape):", cls_embeddings.shape)
print(cls_embeddings)


Sample Texts:
- the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
- the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .
- effective but too-tepid biopic

Output Keys: odict_keys(['last_hidden_state'])
Last Hidden State shape: torch.Size([3, 52, 768])

[CLS] Token Embeddings (shape): torch.Size([3, 768])
tensor([[-0.0332, -0.0168,  0.0194,  ...,  0.0476,  0.5834,  0.3036],
        [-0.2062, -0.0490, -0.4036,  ..., -0.1186,  0.6141,  0.3919],
        [-0.2706, -0.1265, -0.0500,  ..., -0.3721,  0.2477,  0.3306]])


#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [28]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

def extract_cls_embeddings(texts, tokenizer, model, batch_size=32, device="cpu"):
    all_embeddings = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :] 
            all_embeddings.append(cls_embeddings.cpu())

    return torch.cat(all_embeddings).numpy()

model.eval()

train_texts = [example['text'] for example in train_dataset]
val_texts = [example['text'] for example in val_dataset]
test_texts = [example['text'] for example in test_dataset]

train_labels = [example['label'] for example in train_dataset]
val_labels = [example['label'] for example in val_dataset]
test_labels = [example['label'] for example in test_dataset]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

train_features = extract_cls_embeddings(train_texts, tokenizer, model, device=device)
val_features = extract_cls_embeddings(val_texts, tokenizer, model, device=device)
test_features = extract_cls_embeddings(test_texts, tokenizer, model, device=device)

clf = LinearSVC()
clf.fit(train_features, train_labels)

val_preds = clf.predict(val_features)
val_acc = accuracy_score(val_labels, val_preds)
print("Validation Accuracy:", val_acc)
print(classification_report(val_labels, val_preds))

# Test
test_preds = clf.predict(test_features)
test_acc = accuracy_score(test_labels, test_preds)
print("Test Accuracy:", test_acc)
print(classification_report(test_labels, test_preds))

cuda
Validation Accuracy: 0.8217636022514071
              precision    recall  f1-score   support

           0       0.81      0.84      0.83       533
           1       0.84      0.80      0.82       533

    accuracy                           0.82      1066
   macro avg       0.82      0.82      0.82      1066
weighted avg       0.82      0.82      0.82      1066

Test Accuracy: 0.798311444652908
              precision    recall  f1-score   support

           0       0.79      0.81      0.80       533
           1       0.81      0.78      0.80       533

    accuracy                           0.80      1066
   macro avg       0.80      0.80      0.80      1066
weighted avg       0.80      0.80      0.80      1066



-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [30]:
from datasets import load_dataset
from transformers import AutoTokenizer

def tokenize_function(example):
    return tokenizer(example['text'], padding="max_length", truncation=True)

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

print(tokenized_datasets['train'][0])


Map: 100%|████████████████████████████████████████████████████████████████| 1066/1066 [00:00<00:00, 5695.01 examples/s]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1, 'input_ids': [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [1]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased",num_labels=2)

#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(example['text'], padding="max_length", truncation=True)
    
tokenized_datasets = dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased",
    num_labels=2
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.from_numpy(logits).argmax(dim=1).numpy()
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy = "epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate() 

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\leona\AppData\Local\Temp\ipykernel_15124\3278362826.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: leonardodelbene27 (leonardodelbene27-universit-di-fire

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.376700,0.384299,0.823640,0.806394,0.851782,0.828467
2,0.190100,0.426637,0.846154,0.868263,0.816135,0.841393
3,0.105500,0.709590,0.832083,0.822993,0.846154,0.834413


{'eval_loss': 0.4266369938850403,
 'eval_accuracy': 0.8461538461538461,
 'eval_precision': 0.8682634730538922,
 'eval_recall': 0.8161350844277674,
 'eval_f1': 0.8413926499032882,
 'eval_runtime': 23.1712,
 'eval_samples_per_second': 46.005,
 'eval_steps_per_second': 0.734,
 'epoch': 3.0}

-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import CLIPModel, CLIPProcessor
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
from tqdm import tqdm
from torch import nn, optim
import wandb

In [2]:
def text_prompts(labels, model,processor, sketch= False, device="cpu"):
    if sketch:
        text_prompt = [f"a sketch of a {label}" for label in labels]
    else:
        text_prompt = [f"a photo of a {label}" for label in labels]
        
    text_input = processor(text=text_prompt, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        text_features = model.get_text_features(**text_input)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    return text_features

In [3]:
def evaluate(data, model, transform, text_features, device="cpu"):
    correct, total = 0, 0
    for example in tqdm(data, desc="Evaluating"):
        img = example["image"].convert("RGB")
        label = example["label"]

        image = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.get_image_features(image)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            logits = image_features @ text_features.T
            pred = logits.argmax(dim=1).item()

        if pred == label:
            correct += 1
        total += 1

    acc = correct / total * 100
    print(f"\n Accuracy: {acc:.2f}% on {total} samples")


In [4]:
#Valutazione su ImageNette
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "openai/clip-vit-base-patch16"

processor = CLIPProcessor.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name).to(device)
model.eval()

dataset = load_dataset("frgfm/imagenette", "160px")

val_data = dataset["validation"]
id2label = sorted(list(set(val_data['label'])))
label_names_imageNette = [val_data.features["label"].int2str(i) for i in id2label]

transform = transforms.Compose([
                     transforms.Resize((224, 224)),
                     transforms.ToTensor(),
                     transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], 
                                         std=[0.26862954, 0.26130258, 0.27577711])
                     ])


text_features = text_prompts(label_names_imageNette, model,processor, sketch=False, device=device)

evaluate(val_data, model, transform, text_features, device=device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Evaluating: 100%|██████████████████████████████████████████████████████████████████| 3925/3925 [01:30<00:00, 43.27it/s]


 Accuracy: 98.90% on 3925 samples


In [5]:
#Valutazione su immagini sketch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "openai/clip-vit-base-patch16"
processor = CLIPProcessor.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name).to(device)
model.eval()

def add_label_text(example):
    example["label_text"] = label_names[example["label"]]
    return example

ds = load_dataset("kmewhort/tu-berlin-png", split="test")
label_names = ds.features["label"].names
ds = ds.map(add_label_text)

transform_sketch = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                         std=[0.26862954, 0.26130258, 0.27577711])
])

text_features = text_prompts(label_names, model,processor, sketch=True, device= device)

evaluate(ds, model, transform, text_features, device=device)

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 201/201 [00:07<00:00, 25.95it/s]


 Accuracy: 55.22% on 201 samples


In [4]:
def train_fine_tuning(model, train_loader, text_features, opt, loss_fn, num_epochs, device="cpu", project_name="homework-3", run_name="Clip_Fine_tuning"):
    wandb.init(
        project=project_name,
        name=run_name,
        config={
            "epochs": num_epochs,
            "optimizer": opt.__class__.__name__,
            "loss_function": loss_fn.__class__.__name__,
            "device": device,
        }
    )

    for epoch in range(num_epochs):
        total_loss = 0.0
        total = 0
        correct = 0

        for pair in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            img = pair["pixel_values"].to(device)
            labels = pair["labels"].to(device)

            image_features = model.get_image_features(img)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)

            logits = image_features @ text_features.T
            loss = loss_fn(logits, labels)
            preds = logits.argmax(dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)
            acc = correct / total * 100

            loss.backward()
            opt.step()
            opt.zero_grad()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} - Loss: {total_loss:.4f} - Accuracy: {acc:.2f}")
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": total_loss,
            "train_accuracy": acc
        })

    model.save_pretrained("clip_lora_adapter")

def evaluate_fine_tuning(model, dataloader, text_features, device="cpu"):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for pair in tqdm(dataloader, desc="Evaluating"):
            img = pair["pixel_values"].to(device)
            labels = pair["labels"].to(device)

            image_features = model.get_image_features(img)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)

            logits = image_features @ text_features.T
            preds = logits.argmax(dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total * 100
    wandb.log({"eval_accuracy": acc})
    print(f"\nAccuracy: {acc:.2f}% on {total} samples")
    wandb.finish()

In [5]:
#Fine-Tuning con Lo-Ra su dataset di Sketch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def add_label_text(example):
    example["label_text"] = label_names[example["label"]]
    return example

train_data = load_dataset("kmewhort/tu-berlin-png", split="train")
label_names = train_data.features["label"].names
train_data = train_data.map(add_label_text)

test_data = load_dataset("kmewhort/tu-berlin-png", split="test")
test_data = test_data.map(add_label_text)

num_classes = len(label_names)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.48145466, 0.4578275, 0.40821073],
        std=[0.26862954, 0.26130258, 0.27577711]
    )
])

model_name = "openai/clip-vit-base-patch16"
processor = CLIPProcessor.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name)


lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "fc1", "fc2"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION
)

model = get_peft_model(model, lora_config)
model.vision_model = get_peft_model(model.vision_model, lora_config)
model.text_model = get_peft_model(model.text_model, lora_config)

model.to(device)
model.train()
model.print_trainable_parameters()

text_features =text_prompts(label_names, model, processor, sketch=True, device= device)

def collate_fn(batch):
    images = [transform(example["image"].convert("RGB")) for example in batch]
    labels = [example["label"] for example in batch]
    return {
        "pixel_values": torch.stack(images),
        "labels": torch.tensor(labels)
    }

batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

lr = 1e-4
wd = 0.01
num_epochs= 5

opt = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss()

train_fine_tuning(model, train_loader, text_features, opt, loss_fn, num_epochs, device=device)

evaluate_fine_tuning(model, test_loader, text_features, device=device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
C:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
C:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\peft\mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'openai/clip-vit-base-patch16' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


trainable params: 3,932,160 || all params: 153,552,897 || trainable%: 2.5608


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: leonardodelbene27 (leonardodelbene27-universit-di-firenze) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████| 622/622 [40:27<00:00,  3.90s/it]


Epoch 1 - Loss: 3249.4669 - Accuracy: 70.64


...

Epoch 5: 100%|███████████████████████████████████████████████████████████████████████| 622/622 [50:52<00:00,  4.91s/it]


Epoch 5 - Loss: 3172.7818 - Accuracy: 89.99


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.47s/it]



Accuracy: 78.61% on 201 samples


epoch,▁▃▅▆█
eval_accuracy,▁
train_accuracy,▁▄▆▇█
train_loss,█▅▃▂▁
epoch,5
eval_accuracy,78.60697
train_accuracy,89.98944
train_loss,3172.78176


#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.